In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np

ModuleNotFoundError: No module named 'torch'

In [0]:
import torchvision
import torchvision.transforms as transforms

In [8]:
BATCH_SIZE = 64

transform = transforms.Compose([transforms.ToTensor()])

trainset = torchvision.datasets.MNIST(root='./data',
                                      train=True,
                                      download=True,
                                      transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size = BATCH_SIZE,
                                          shuffle=True,
                                          num_workers=2)

testset = torchvision.datasets.MNIST(root='./data',
                                      train=False,
                                      download=True,
                                      transform=transform)
testloader = torch.utils.data.DataLoader(testset,
                                          batch_size = BATCH_SIZE,
                                          shuffle=True,
                                          num_workers=2)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


In [0]:
N_STEPS = 28
N_INPUTS = 28
N_NEURONS = 150
N_OUTPUTS = 10
N_EPHOCS = 10

In [0]:
class ImageRNN(nn.Module):
    def __init__(self, batch_size, n_steps, n_inputs, n_neurons, n_outputs):
        super(ImageRNN, self).__init__()
        
        self.n_neurons = n_neurons
        self.batch_size = batch_size
        self.n_steps = n_steps
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        
        self.basic_rnn = nn.RNN(self.n_inputs, self.n_neurons) 
        
        self.FC = nn.Linear(self.n_neurons, self.n_outputs)
        
    def init_hidden(self,):
        # (num_layers, batch_size, n_neurons)
        return (torch.zeros(1, self.batch_size, self.n_neurons))
        
    def forward(self, X):
        # transforms X to dimensions: n_steps X batch_size X n_inputs
        X = X.permute(1, 0, 2) 
        
        self.batch_size = X.size(1)
        self.hidden = self.init_hidden()
        
        lstm_out, self.hidden = self.basic_rnn(X, self.hidden)      
        out = self.FC(self.hidden)
        
        return out.view(-1, self.n_outputs) # batch_size X n_output

In [36]:
dataiter = iter(trainloader)
images, labels = dataiter.next()
model = ImageRNN(BATCH_SIZE, N_STEPS, N_INPUTS, N_NEURONS, N_OUTPUTS)
logits = model(images.view(-1, 28,28))
print(logits[0:10])

tensor([[ 0.0218,  0.0550,  0.0033,  0.0166, -0.0872,  0.0466,  0.0780, -0.1358,
          0.0536, -0.1956],
        [ 0.0202,  0.0579,  0.0262,  0.0347, -0.0921,  0.0566,  0.0796, -0.1495,
          0.0497, -0.1967],
        [-0.0167,  0.0556,  0.0581, -0.0241, -0.1004,  0.0751,  0.1060, -0.1290,
          0.1109, -0.1863],
        [ 0.0126,  0.0427,  0.0079,  0.0168, -0.0933,  0.0513,  0.0636, -0.1292,
          0.0504, -0.1959],
        [ 0.0119,  0.0399,  0.0178,  0.0336, -0.0979,  0.0456,  0.0613, -0.1324,
          0.0472, -0.1916],
        [ 0.0124,  0.0484,  0.0762,  0.0641, -0.0913,  0.0765,  0.0748, -0.1413,
          0.0597, -0.1717],
        [ 0.0283,  0.0560,  0.0267,  0.0282, -0.1044,  0.0623,  0.0831, -0.1523,
          0.0575, -0.1982],
        [ 0.0252,  0.0352,  0.0026,  0.0151, -0.0904,  0.0443,  0.0713, -0.1407,
          0.0521, -0.2064],
        [-0.0059,  0.0410,  0.0391, -0.0107, -0.0899,  0.0650,  0.0869, -0.1448,
          0.0771, -0.1907],
        [ 0.0043,  

In [0]:
import torch.optim as optim

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Model instance
model = ImageRNN(BATCH_SIZE, N_STEPS, N_INPUTS, N_NEURONS, N_OUTPUTS)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def get_accuracy(logit, target, batch_size):
    ''' Obtain accuracy for training round '''
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects/batch_size
    return accuracy.item()

In [38]:
for epoch in range(N_EPHOCS):
    train_running_loss = 0.0
    train_acc = 0.0
    model.train()
    i = 0
    for inputs, labels in trainloader:
        optimizer.zero_grad()
        model.hidden = model.init_hidden()
        
        inputs = inputs.view(-1, 28, 28)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_running_loss += loss.detach().item()
        train_acc += get_accuracy(outputs, labels, BATCH_SIZE)
        i+=1

    model.eval()
    print('Epoch:  %d | Loss: %.4f | Train Accuracy: %.2f' %(epoch, train_running_loss / i, train_acc/i))

Epoch:  0 | Loss: 0.7693 | Train Accuracy: 74.15
Epoch:  1 | Loss: 0.3524 | Train Accuracy: 89.60
Epoch:  2 | Loss: 0.2573 | Train Accuracy: 92.57
Epoch:  3 | Loss: 0.2065 | Train Accuracy: 94.07
Epoch:  4 | Loss: 0.1794 | Train Accuracy: 94.84
Epoch:  5 | Loss: 0.1585 | Train Accuracy: 95.50
Epoch:  6 | Loss: 0.1471 | Train Accuracy: 95.77
Epoch:  7 | Loss: 0.1363 | Train Accuracy: 96.13
Epoch:  8 | Loss: 0.1297 | Train Accuracy: 96.31
Epoch:  9 | Loss: 0.1195 | Train Accuracy: 96.51


In [0]:
test_acc = 0.0
for i, data in enumerate(testloader, 0):
    inputs, labels = data
    inputs = inputs.view(-1, 28, 28)

    outputs = model(inputs)

    test_acc += get_accuracy(outputs, labels, BATCH_SIZE)

In [41]:
print('Test Accuracy: %.2f'%( test_acc/i))

Test Accuracy: 96.04
